In [1]:
import os
import json
import sys
sys.path.append("transformers/src")
import csv
from sentence_transformers import SentenceTransformer, util
import random
import datetime
import requests
from tweet_graph import *
import pickle as pkl
from transformers import LongformerModel, LongformerConfig
from transformers import RobertaTokenizer
import torch
import torch.nn.functional as F
from tqdm import tqdm
import pickle as pkl
from collections import Counter
import math
from deep_rank import *
import warnings
warnings.filterwarnings("ignore")
csv.field_size_limit(sys.maxsize)


131072

In [2]:
data_dir="data"
tweet_collection="dehydrate_tweet_collection.json"
user_profile="dehydrate_user_profile.json"
english_news="english_news.csv"
news_collection="news_collection.json"
news_tweet_relation="news_tweet_relation.json"
tweet_tweet_relation="tweet_tweet_relation.json"

In [3]:
# 加载数据
def load_data():
    #加载推文的数据:推文->用户     
    tweet_map={}
    with open(os.path.join(data_dir,tweet_collection)) as f:
        for line in f:
            line=line.strip()
            line=json.loads(line)
            if line["tweet_id"] is None or line["user_id"] is None:
                continue
                
            #生成随机的时间戳            
            timestamp = random.uniform(datetime.datetime.now().timestamp()-50000000, datetime.datetime.now().timestamp())
            date_time = datetime.datetime.fromtimestamp(timestamp)
            tweet_map[line["tweet_id"]]={"user_id":line["user_id"],"time":date_time}
    print("收集推文数",len(tweet_map))
                
    #加载用户的信息数据   
    # ['user_id', 'recent_tweet_ids']
    user_profile_data={}
    with open(os.path.join(data_dir,user_profile)) as f:
        for line in f:
            line=line.strip()
            line=json.loads(line)
            user_profile_data[line["user_id"]]=line["recent_tweet_ids"]
    print("收集用户数",len(user_profile_data))
    
    #加载新闻数据信息
    #表头为：news_id,agency,claim,fact_url,label,lang,ref_source,ref_source_url,statement     
    en_news_data={}
    with open(os.path.join(data_dir,english_news), newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        header=next(spamreader)
        for row in spamreader:
            try:
                news_id=row[header.index("news_id")]
                label=row[header.index("label")]
                #推文             
                claim=row[header.index("claim")]
                ref_source=json.loads(row[header.index("ref_source")])
                title=ref_source["title"]
                text=ref_source["text"]
                top_img=ref_source["top_img"]
                en_news_data[news_id]={"claim":claim,"label":label,"ref_source":ref_source,"title":title,"text":text,"top_img":top_img}
            except Exception as e:
                continue 
    print("收集新闻数",len(en_news_data))
    
    #加载推文与新闻的关系   
    tweet_news_data={}
    with open(os.path.join(data_dir,news_tweet_relation)) as f:
        for line in f:
            line=line.strip()
            line=json.loads(line)
            news_id=line["news_id"]
            tweet_list=line["tweet_list"]
            for item in tweet_list:
                try:
                    tweet_id=item["$numberLong"]
                    tweet_news_data[tweet_id]=news_id
                except Exception as e:
                    continue 
    print("收集推特与新闻的关系",len(tweet_news_data))
    return tweet_map,user_profile_data,en_news_data,tweet_news_data

In [17]:
#第一次执行即可，不必每次都要执行
tweet_map,user_profile_data,en_news_data,tweet_news_data=load_data()
pkl.dump(tweet_map,open("save/tweet_map.pkl","wb"))
pkl.dump(user_profile_data,open("save/user_profile_data.pkl","wb"))
pkl.dump(en_news_data,open("save/en_news_data.pkl","wb"))
pkl.dump(tweet_news_data,open("save/tweet_news_data.pkl","wb"))


收集推文数 8867495
收集用户数 951
收集新闻数 2696
收集推特与新闻的关系 109883


In [4]:
tweet_map=pkl.load(open("save/tweet_map.pkl","rb"))
user_profile_data=pkl.load(open("save/user_profile_data.pkl","rb"))
en_news_data=pkl.load(open("save/en_news_data.pkl","rb"))
tweet_news_data=pkl.load(open("save/tweet_news_data.pkl","rb"))

In [5]:
def similarity_news_text(s1,s2,model):
    # random_number = random.random()
    # return random_number
    # model = LongformerModel.from_pretrained("models/longformer/longformer-base-4096")
    tokenizer = RobertaTokenizer.from_pretrained('models/roberta-base')
    tokenizer.model_max_length = model.config.max_position_embeddings
    
    input_ids = torch.tensor(tokenizer.encode([s1])).unsqueeze(0)  # batch of size 1
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device) # initialize to local attention
    input_ids, attention_mask = pad_to_window_size(
            input_ids, attention_mask, model.config.attention_window[0], tokenizer.pad_token_id)
    output1 = model(input_ids, attention_mask=attention_mask)[0]
    
    input_ids = torch.tensor(tokenizer.encode([s2])).unsqueeze(0)  # batch of size 1
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device) # initialize to local attention
    input_ids, attention_mask = pad_to_window_size(
            input_ids, attention_mask, model.config.attention_window[0], tokenizer.pad_token_id)
    output2 = model(input_ids, attention_mask=attention_mask)[0]
    cos_sim = util.cos_sim(output1.view(-1), output2.view(-1))
    return cos_sim

In [6]:
# SentenceTransformer 计算推文的相似度
def similarity_posts(s1,s2,model):
    # model = SentenceTransformer('models/all-roberta-large-v1')
    # random_number = random.random()
    # return random_number
    emb1 = model.encode(s1)
    emb2 = model.encode(s2)
    cos_sim = util.cos_sim(emb1, emb2)
    return cos_sim

In [7]:
# 计算新闻文章的相似度图像
def similarity_news_visual(img_i,img_j,model):
    # random_number = random.random()
    # return random_number
    # model = deep_rank_model()
    # model.load_weights("models/Deep-Ranking/deepranking-v2-150000.h5")
    image1 = load_img(img_i)
    image1 = img_to_array(image1).astype("float64")
    image1 = transform.resize(image1, (224, 224))
    image1 *= 1. / 255
    image1 = np.expand_dims(image1, axis = 0)
    embedding1 = model.predict([image1, image1, image1])[0]
    image2 =load_img(img_j)
    image2 = img_to_array(image2).astype("float64")
    image2 = transform.resize(image2, (224, 224))
    image2 *= 1. / 255
    image2 = np.expand_dims(image2, axis = 0)
    embedding2 = model.predict([image2,image2,image2])[0]
    cos_sim = util.cos_sim(embedding1, embedding2)
    return cos_sim

In [8]:
class edge(object):
    """边"""
    def __init__(self):
        super(edge, self).__init__()
        self.in_node=None
        self.out_node=None
        self.type=None
        self.weight=None

In [11]:
class tweet_graph(object):
    """推特推文图."""
    def __init__(self):
        super(tweet_graph, self).__init__()
        #定义节点         
        self.tweet_nodes=[]
        self.edges=[]
        self.news=[]
        #加载3个模型         
        self.SentenceTransformerModel = SentenceTransformer('models/all-roberta-large-v1')
        self.LongformerModel = LongformerModel.from_pretrained("models/longformer/longformer-base-4096")
        self.DeepRankModel = deep_rank_model()
        self.DeepRankModel.load_weights("models/Deep-Ranking/deepranking-v2-150000.h5")
        
    def build_graph(self,tweet_map,user_profile_data,en_news_data,tweet_news_data):
        self.tweet_map=tweet_map
        self.tweet_news_data=tweet_news_data
        self.en_news_data=en_news_data
        
        #初始化节点
        with open("save/tweet_nodes.csv","w") as f:
            for tweet_id,node_content in tweet_map.items():
                #保留引用新闻的推特
                if tweet_id in tweet_news_data:
                    news_id_i=tweet_news_data[tweet_id]
                    # if news_id_i not in en_news_data or  not os.path.isfile(os.path.join("data","news_img",news_id_i+".jpg")):
                    if news_id_i not in en_news_data:
                        continue
                    self.news.append(tweet_news_data[tweet_id])
                    self.tweet_nodes.append(tweet_id)
                    
                    
        #选择一部分节点,100个
        # self.tweet_nodes=self.tweet_nodes
        self.tweet_nodes=random.sample(self.tweet_nodes, 10000)
        print("影响图有节点个数:",len(self.tweet_nodes))
        
        #开始构造边
        edges_count=0
        with open("save/edges.csv","w") as f:
            for node_i in tqdm(range(len(self.tweet_nodes))):
                for node_j in range(node_i+1,len(self.tweet_nodes)):
                    #计算两个节点的时间   
                    node_i_t=tweet_map[self.tweet_nodes[node_i]]["time"]
                    node_j_t=tweet_map[self.tweet_nodes[node_j]]["time"]
                    node_i_u=tweet_map[self.tweet_nodes[node_i]]["user_id"]
                    node_j_u=tweet_map[self.tweet_nodes[node_j]]["user_id"]
                    node_i_a=self.news[node_i]
                    node_j_a=self.news[node_j]
                    if node_i_t<node_j_t:
                        e=edge()
                        e.in_node=self.tweet_nodes[node_i]
                        e.out_node=self.tweet_nodes[node_j]
                        if node_i_u != node_j_u:
                            e.type="External"
                        elif node_i_a!=node_j_a :
                            e.type="Internal"
                        else:
                            continue
                        self.edges.append(e)
                        f.write(str(node_i)+","+str(node_j)+","+e.type)
                        edges_count+=1
        pkl.dump(self.edges,open("save/edges.pkl","wb"))
        
        self.edges=pkl.load(open("save/edges.pkl","rb"))
        print("影响图有边个数--:",len(self.edges))
        
        self.set_weight()
        
    #计算时间差   
    def get_detal_t(self,e_in,e_out):
        node_i_t=self.tweet_map[e_in]["time"]
        node_j_t=self.tweet_map[e_out]["time"]
        detal_t=(node_j_t-node_i_t).total_seconds()//3600
        value_detal_t=math.exp(1-detal_t)
        return value_detal_t
    
    #计算相似度推文之间的
    def get_similarity_cij(self,e_in,e_out):
        node_i_c=self.en_news_data[self.tweet_news_data[e_in]]["claim"]
        node_j_c=self.en_news_data[self.tweet_news_data[e_out]]["claim"]
        sim_cij=similarity_posts(node_i_c,node_j_c,self.SentenceTransformerModel)
        return sim_cij
    
    #计算新闻之间的相似度文本+图像特征
    def get_similarity_aij(self,e_in,e_out):
        news_id_i=self.tweet_news_data[e_in]
        news_id_j=self.tweet_news_data[e_out]
        
        #计算文本
        node_i_a_text=self.en_news_data[news_id_i]["text"]
        node_j_a_text=self.en_news_data[news_id_j]["text"]
        sim_aij_text=similarity_news_text(node_i_a_text,node_j_a_text,self.LongformerModel)
        
        #计算图像         
        img_i_file=os.path.join("data","news_img",news_id_i+".jpg")
        img_j_file=os.path.join("data","news_img",news_id_j+".jpg")
        
        sim_vij_vis=0
        if os.path.isfile(img_i_file) and os.path.isfile(img_j_file):
            sim_vij_vis=similarity_news_visual(img_i_file,img_j_file,self.DeepRankModel)
        
        mu=0.8
        sim_score=mu*sim_aij_text+(1-mu)*sim_vij_vis
        return sim_score
            
    #计算权重     
    def set_weight(self):
        # 计算权重
        print("计算权重")
        for e in tqdm(self.edges):
            e_in=e.in_node
            e_out=e.out_node
            
            #时间差             
            value_detal_t=self.get_detal_t(e_in,e_out)
            
            #推文的相似度             
            sim_cij=self.get_similarity_cij(e_in,e_out)
            
            #新闻的相似度             
            sim_aij=self.get_similarity_aij(e_in,e_out)
            
            e.weight=value_detal_t*sim_cij*sim_aij
        
        pkl.dump(self.edges,open("save/edges.pkl","wb"))

In [12]:
tg=tweet_graph()
tg.build_graph(tweet_map,user_profile_data,en_news_data,tweet_news_data)

Some weights of LongformerModel were not initialized from the model checkpoint at models/longformer/longformer-base-4096 and are newly initialized: ['encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.4.output.LayerNorm.bias', 'encoder.layer.3.attention.self.value.bias', 'encoder.layer.6.attention.self.query.weight', 'encoder.layer.3.output.dense.bias', 'encoder.layer.9.attention.self.value_global.weight', 'encoder.layer.4.output.LayerNorm.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.2.attention.output.dense.weight', 'encoder.layer.9.attention.self.value.bias', 'encoder.layer.6.attention.output.LayerNorm.bias', 'encoder.layer.3.output.dense.weight', 'encoder.layer.10.attention.self.key.bias', 'encoder.layer.9.attention.self.value_global.bias', 'encoder.layer.4.attention.self.key_global.bias', 'encoder.layer.6.attention.output.LayerNorm.weight', 'encoder.layer.2.attention.self.value_global.weight', 'encoder.layer.2.attention.self.query_global.weight', 'enco

影响图有节点个数: 10000


100%|██████████| 10000/10000 [02:36<00:00, 63.93it/s]


影响图有边个数--: 24957578
计算权重


100%|██████████| 24957578/24957578 [07:15<00:00, 57277.59it/s]


In [ ]:
#开始分析影响力图